In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [26]:
Train= pd.read_csv('Data/Train.csv')
Test= pd.read_csv('Data/Test.csv')

In [27]:
#remove all nul values from train and test
Train=Train.dropna()


In [34]:
#for test data fill null values with mean
Test=Test.fillna(Test.mean())


In [36]:
Test.head()

,RecordID,hospital_id,icu_id,age,elective_surgery,pre_icu_los_days,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,gcs_eyes_apache,...,apache_2_bodysystem_Cardiovascular,apache_2_bodysystem_Gastrointestinal,apache_2_bodysystem_Haematologic,apache_2_bodysystem_Metabolic,apache_2_bodysystem_Neurologic,apache_2_bodysystem_Renal/Genitourinary,apache_2_bodysystem_Respiratory,apache_2_bodysystem_Trauma,apache_2_bodysystem_Undefined Diagnoses,apache_2_bodysystem_Undefined diagnoses
0,50001,267,1658,61.0,0,0.000000,302.0,105.02,0,4.0,...,1,0,0,0,0,0,0,0,0,0
1,50002,161,1487,80.0,0,0.025000,121.0,401.01,0,1.0,...,0,0,0,0,1,0,0,0,0,0
2,50003,287,1845,79.0,0,0.792361,117.0,106.01,0,4.0,...,1,0,0,0,0,0,0,0,0,0
3,50004,170,1373,51.0,0,0.060417,301.0,403.01,0,2.0,...,0,0,0,0,1,0,0,0,0,0
4,50005,276,1747,56.0,0,0.069444,108.0,203.01,0,3.0,...,0,0,0,0,0,0,1,0,0,0


In [37]:
#apply one hot encoding
Train=pd.get_dummies(Train)
Test=pd.get_dummies(Test)

In [38]:
X=Train.loc[:,Train.columns!='hospital_death']
y=Train['hospital_death']

In [39]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3)

In [40]:
from sklearn.metrics import roc_auc_score
from sklearn.naive_bayes import GaussianNB

# Fit the model on the transformed X_train
model = GaussianNB()
model.fit(X_train, y_train)

# Predict and evaluate on the transformed X_test
y_pred = model.predict(X_test)
print(roc_auc_score(y_test, y_pred))


0.7335166950669745


In [41]:
# Predict and evaluate on the transformed X_test
hospital_death = model.predict(Test)

# Create a DataFrame for the predictions
predictions_df = pd.DataFrame(hospital_death, columns=['hospital_death'])

# Add the record ID from the test data to the predictions DataFrame
predictions_df.insert(0, 'RecordID', Test['RecordID'])

# Save the predictions to a CSV file
predictions_df.to_csv('predictionsNAIVEBAYES.csv', index=False)

In [42]:
predictions_df.head()

,RecordID,hospital_death
0,50001,0
1,50002,1
2,50003,0
3,50004,0
4,50005,0


In [1]:
import pandas as pd
%pip install mlxtend
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

Note: you may need to restart the kernel to use updated packages.


In [4]:
import numpy as np
Train= pd.read_csv('Data/Train.csv')
Test= pd.read_csv('Data/Test.csv')

In [5]:
binary_cols = ['elective_surgery', 'apache_post_operative', 'gcs_unable_apache', 'intubated_apache', 'ventilated_apache','immunosuppression', 'solid_tumor_with_metastasis']
len(binary_cols)

7

In [6]:
#fill null values with mode in binrary_cols
for col in binary_cols:
    Train[col].fillna(Train[col].mode()[0], inplace=True)
    Test[col].fillna(Test[col].mode()[0], inplace=True)

In [7]:
#fill all numerics with knn imputer
numeric_cols = [col for col in Train.select_dtypes(include=[np.number]).columns if col not in binary_cols]
numeric_colsTest = [col for col in Test.select_dtypes(include=[np.number]).columns if col not in binary_cols]
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=3)
Train[numeric_cols] = imputer.fit_transform(Train[numeric_cols])
#do same for test
Test[numeric_colsTest] = imputer.fit_transform(Test[numeric_colsTest])

In [28]:
# make categorical data that has all dtypes objects and columns in binary_cols
categorical_cols = [col for col in Train.columns if Train[col].dtype == 'object' or col in binary_cols]
categorical_colsTest = [col for col in Test.columns if Test[col].dtype == 'object' or col in binary_cols]


In [34]:
#make a data fram for categorical data
Train_cat = Train[categorical_cols]
Test_cat = Test[categorical_colsTest]

In [35]:
Train_cat.shape

(50000, 14)

In [36]:
#full in null values with mode of Train_cat
for col in categorical_cols:
    Train_cat[col].fillna(Train_cat[col].mode()[0], inplace=True)
    Test_cat[col].fillna(Test_cat[col].mode()[0], inplace=True)

C:\Users\hamza\AppData\Local\Temp\ipykernel_1964\1933264251.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Train_cat[col].fillna(Train_cat[col].mode()[0], inplace=True)
C:\Users\hamza\AppData\Local\Temp\ipykernel_1964\1933264251.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Test_cat[col].fillna(Test_cat[col].mode()[0], inplace=True)
C:\Users\hamza\AppData\Local\Temp\ipykernel_1964\1933264251.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi

In [168]:
#define x as all columns except target
X=Train.loc[:,Train.columns!='hospital_death']
y=Train['hospital_death']

In [169]:
#now run categoricalNB on categorical_cols 
from sklearn.naive_bayes import CategoricalNB
from sklearn.model_selection import train_test_split

# Split the data into train and test sets with a test size of 0.3
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)




In [48]:
from sklearn.preprocessing import LabelEncoder

In [64]:
#i want to label encoding on all my cateogircal data
for col in categorical_cols:
    le = LabelEncoder()
    Train[col] = le.fit_transform(Train[col])
    Test[col] = le.fit_transform(Test[col])

In [118]:
Test.head()

,RecordID,hospital_id,icu_id,ethnicity,gender,icu_admit_source,icu_stay_type,icu_type,apache_3j_bodysystem,apache_2_bodysystem,...,h1_sysbp_max,h1_sysbp_min,h1_sysbp_noninvasive_max,h1_sysbp_noninvasive_min,d1_glucose_max,d1_potassium_max,apache_4a_hospital_death_prob,apache_4a_icu_death_prob,immunosuppression,solid_tumor_with_metastasis
0,50001.0,267.0,1658.0,2,0,0,0,0,0,0,...,167.0,121.0,167.0,121.0,117.0,4.4,0.02,0.01,0,0
1,50002.0,161.0,1487.0,5,1,0,0,6,7,4,...,159.0,94.0,159.0,94.0,136.0,3.5,0.85,0.80,0,0
2,50003.0,287.0,1845.0,2,1,1,2,1,0,0,...,126.0,108.0,126.0,108.0,228.0,4.2,0.15,0.07,0,0
3,50004.0,170.0,1373.0,5,0,0,0,5,7,4,...,142.0,127.0,142.0,127.0,443.0,3.2,0.09,0.04,0,0
4,50005.0,276.0,1747.0,0,0,0,0,0,8,6,...,110.0,110.0,110.0,110.0,219.0,3.6,0.13,0.07,0,0


In [170]:
from sklearn.neighbors import KNeighborsClassifier

fwd_feature_selectorKNN = SFS(KNeighborsClassifier(), k_features=30, forward=True,
                                                  floating=False, verbose=2, scoring='roc_auc', cv=5).fit(X_train, y_train)


[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:  3.3min

[2023-09-27 00:08:34] Features: 1/30 -- score: 0.7420351213824349[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:  1.1min

[2023-09-27 00:10:05] Features: 2/30 -- score: 0.7401242801837583[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:  1.5min

[2023-09-27 00:12:04] Features: 3/30 -- score: 0.7454191789438994[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:  1.2min

[2023-09-27 00:13:39] Features: 4/30 -- score: 0.7471757782001053[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:  1.5min

[2023-09-27 00:15:37] Features: 5/30 -- score: 0.7410796857395534[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:  1.8min

[2023-09-27 00:17:59] Features: 6/30 -- score: 0.7436942870234804[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:  1.9min

[2023-09-27 00:20:31] Features: 7/30 -- score: 0.7441514824197748[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:  2.1min

[2023-09-27 00:23:11] Features: 8/30 -- score: 0.

In [86]:
from sklearn.naive_bayes import GaussianNB

fwd_feature_selector = SFS(GaussianNB(), k_features=30, forward=True, floating=False, verbose=2, scoring='roc_auc', cv=5).fit(X_train, y_train)

                           

[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    3.9s

[2023-09-26 22:47:10] Features: 1/30 -- score: 0.8370579774818193[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    6.0s

[2023-09-26 22:47:18] Features: 2/30 -- score: 0.8357120599842878[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    6.2s

[2023-09-26 22:47:27] Features: 3/30 -- score: 0.834779488821044[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    6.3s

[2023-09-26 22:47:36] Features: 4/30 -- score: 0.8344378957343563[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    6.7s

[2023-09-26 22:47:45] Features: 5/30 -- score: 0.8337798644522489[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    6.8s

[2023-09-26 22:47:54] Features: 6/30 -- score: 0.8321221857151467[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    7.2s

[2023-09-26 22:48:03] Features: 7/30 -- score: 0.8303765230798937[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    7.2s

[2023-09-26 22:48:12] Features: 8/30 -- score: 0.8

In [80]:
from sklearn.tree import DecisionTreeClassifier
fwd_feature_selector = SFS(DecisionTreeClassifier(), k_features=(10,40), forward=True,
                                                  floating=False, verbose=2, scoring='roc_auc', cv=5).fit(X_train, y_train)
#cv is croos validation 5 times
#The cv=5 parameter means that this process is done using 5-fold cross-validation on your training data (X_train and y_train). 
# In each fold of the cross-validation, 4/5 of the data is used for training and 1/5 is used for validation.
# The ROC AUC score is averaged over the 5 folds to give a more robust estimate of the model’s performance.
                           

[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:   15.6s

[2023-09-26 17:06:24] Features: 1/40 -- score: 0.8383395704744482[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:   20.8s

[2023-09-26 17:06:53] Features: 2/40 -- score: 0.834362830843793[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:   23.0s

[2023-09-26 17:07:26] Features: 3/40 -- score: 0.8282239832526441[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:   25.3s

[2023-09-26 17:07:59] Features: 4/40 -- score: 0.8212378013308704[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:   26.5s

[2023-09-26 17:08:34] Features: 5/40 -- score: 0.8113648068895924[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:   29.7s

[2023-09-26 17:09:12] Features: 6/40 -- score: 0.8032718307929724[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:   31.2s

[2023-09-26 17:09:51] Features: 7/40 -- score: 0.7907676817909873[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:   35.1s

[2023-09-26 17:10:35] Features: 8/40 -- score: 0.7

In [87]:
fwd_feature_selector.k_feature_names_     # to get the final set of features


('hospital_id',
 'icu_id',
 'ethnicity',
 'gender',
 'icu_admit_source',
 'icu_type',
 'apache_3j_bodysystem',
 'apache_2_bodysystem',
 'age',
 'pre_icu_los_days',
 'apache_3j_diagnosis',
 'gcs_unable_apache',
 'gcs_verbal_apache',
 'heart_rate_apache',
 'resprate_apache',
 'ventilated_apache',
 'd1_heartrate_max',
 'd1_mbp_noninvasive_min',
 'd1_spo2_min',
 'd1_sysbp_noninvasive_min',
 'd1_temp_min',
 'h1_diasbp_min',
 'h1_mbp_max',
 'h1_resprate_max',
 'h1_resprate_min',
 'h1_spo2_max',
 'h1_sysbp_max',
 'd1_potassium_max',
 'apache_4a_hospital_death_prob',
 'apache_4a_icu_death_prob')

In [120]:
from sklearn.metrics import roc_auc_score

# Transform X_test to contain only the selected features
X_test_transformed = fwd_feature_selector.transform(X_test)

# Fit the model on the transformed X_train
# model = DecisionTreeClassifier(max_depth=2, min_samples_split= 10)
modelPRED = GaussianNB(var_smoothing=1e-8)
modelPRED.fit(fwd_feature_selector.transform(X_train), y_train)
y_pred = modelPRED.predict_proba(X_test_transformed)[:, 1]

# Predict and evaluate on the transformed X_test
print(roc_auc_score(y_test, y_pred))


0.8422528857820109


In [192]:
from sklearn.metrics import roc_auc_score

# Transform X_test to contain only the selected features
X_test_transformed = fwd_feature_selectorKNN.transform(X_test)

# Fit the model on the transformed X_train
# model = DecisionTreeClassifier(max_depth=2, min_samples_split= 10)
modelPRED = KNeighborsClassifier()
modelPRED.fit(fwd_feature_selectorKNN.transform(X_train), y_train)
y_pred = modelPRED.predict_proba(X_test_transformed)[:, 1]

# Predict and evaluate on the transformed X_test
print(roc_auc_score(y_test, y_pred))


0.7115551389161391


In [146]:
#define x as columns in columns_of_fwdDECTREE
X=Train[columns_of_fwdDECTREE]
y=Train['hospital_death']

In [147]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)


In [163]:
from sklearn.metrics import roc_auc_score

# Select only the columns in columns_of_fwdDECTREE

# Transform X_test to contain only the selected features
# X_test_transformed = fwd_feature_selector.transform(X_test_selected)

# Fit the model on the transformed X_train
model = DecisionTreeClassifier(max_depth=6, min_samples_split= 2)
model.fit(X_train, y_train)

# Predict and evaluate on the transformed X_test
y_pred = model.predict_proba(X_test)[:, 1]
print(roc_auc_score(y_test, y_pred))


0.8546856540427444


In [162]:
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV

# Define the parameter grid to search over
param_grid = {'max_depth': [2, 4, 6, 8, 10], 'min_samples_split': [2, 5, 10, 20, 50]}

# Create a grid search object
grid_search = GridSearchCV(DecisionTreeClassifier(), param_grid, cv=5, scoring='roc_auc')

# Fit the grid search object on the training data
grid_search.fit(X_train, y_train)

# Get the best parameters and the best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

# Print the best parameters and the best score
print("Best parameters: ", best_params)
print("Best score: ", best_score)

# Fit the model on the transformed X_train using the best parameters
model = DecisionTreeClassifier(max_depth=best_params['max_depth'], min_samples_split=best_params['min_samples_split'])
model.fit(X_train, y_train)

# Predict and evaluate on the transformed X_test
y_pred = model.predict_proba(X_test)[:, 1]
print(roc_auc_score(y_test, y_pred))


Best parameters:  {'max_depth': 6, 'min_samples_split': 50}
Best score:  0.8414081312350647
0.8552767542250399


In [164]:
# Predict and evaluate on the transformed X_test
X_test_transformedTest = fwd_feature_selector.transform(Test)
hospital_death_prob = modelPRED.predict_proba(X_test_transformedTest)

# The predicted probabilities of each class are stored in the second column of the output array
hospital_death = hospital_death_prob[:, 1]


In [165]:
# Predict and evaluate on the transformed X_test
hospital_death_prob = model.predict_proba(Test[columns_of_fwdDECTREE]
)

# The predicted probabilities of each class are stored in the second column of the output array
hospital_death = hospital_death_prob[:, 1]


In [193]:
# Predict and evaluate on the transformed X_test
X_test_transformedTestKNN = fwd_feature_selectorKNN.transform(Test)
hospital_death_prob = modelPRED.predict_proba(X_test_transformedTestKNN)

# The predicted probabilities of each class are stored in the second column of the output array
hospital_death = hospital_death_prob[:, 1]


In [194]:
hospital_death.shape

(30000,)

In [195]:
# Create a DataFrame for the predictions
predictions_df = pd.DataFrame(hospital_death, columns=['hospital_death'])

# Add the record ID from the test data to the predictions DataFrame
predictions_df.insert(0, 'RecordID', Test['RecordID'])

# Save the predictions to a CSV file
predictions_df.to_csv('predictionsFWDSLEECTION.csv', index=False)

In [196]:
predictions_df.shape

(30000, 2)

In [131]:
columns_of_fwdDECTREE=['gender', 'icu_stay_type', 'elective_surgery', 'apache_post_operative', 'gcs_unable_apache', 'intubated_apache', 'ventilated_apache', 'apache_4a_hospital_death_prob', 'immunosuppression', 'solid_tumor_with_metastasis']
